# Problem 1


In [ ]:
import uaibot as ub
import numpy as np
from uaibot.utils import Utils
from uaibot.simobjects import Ball, Frame


kuka = ub.robot.Robot.create_kuka_kr5()
target_pos = Ball(radius=0.05)
target_ori = Frame()
sim = ub.Simulation.create_sim_grid([kuka, target_pos, target_ori])
target_htms = [Utils.htm_rand(trn_min=[-0.7,-0.7, 0], trn_max = [0.7,0.7,0.7]) for _ in range(50)]

q_max = kuka.joint_limit[:, 1]
q_min = kuka.joint_limit[:, 0]

dt = 0.01
K = 5*np.eye(6)

j = 0
old_r = np.array([np.inf] * 6).reshape(-1, 1)
max_iter = int(5 / dt)

fails = []

for n, H_target in enumerate(target_htms):
    print(f"{n}-th target")
    r = np.inf
    i = 0
    target_pos.add_ani_frame(time=j * dt, htm=H_target)
    target_ori.add_ani_frame(time=j * dt, htm=H_target)
    kuka.add_ani_frame(time=j * dt, q=kuka.q0)
    k = 0
    while np.linalg.norm(r) > 1e-2:
        if i > max_iter:
            print("Max iterations reached")
            fails.append(n)
            break
        q = kuka.q.copy()
        r, Jr = kuka.task_function(H_target)
        q_dot = -Utils.dp_inv(Jr) @ K @ r

        if (np.linalg.norm(old_r - r) <= 1e-3) and (np.linalg.norm(r) > 1e-1):
            k += 1
        else:
            k = 0 

        if k > 100:
            print("No improvement for 100 iterations")
            fails.append(n)
            break
        q_ = q + q_dot * dt

        violation_idxs = np.nonzero((q_ > q_max) | (q_ < q_min))[0]
        q_dot[violation_idxs] = 0

        q += q_dot * dt
        i += 1
        j += 1
        old_r = r

        kuka.add_ani_frame(time=j * dt, q=q)

# sim.run()
print(f"Strategy worked on {(len(target_htms) - len(fails))}/{len(target_htms)} targets ({(len(target_htms) - len(fails)) / len(target_htms)}%)")

# Problem 2

In [ ]:
import sys
import uaibot as ub
import numpy as np
from uaibot.utils import Utils
from uaibot.simobjects import Ball, Frame, PointCloud
from scipy.linalg import expm

def R_theta(theta, axis, type_='htm'):
    """Returns rotation matrix around axis by angle theta"""
    Q = np.eye(3) + np.sin(theta) * Utils.S(axis) + (1 - np.cos(theta)) * (Utils.S(axis) @ Utils.S(axis))
    if type_ == 'htm':
        H = np.eye(4)
        H[:3, :3] = Q
        H[:3, 3] = np.zeros(3)
        return np.array(H)
    else:
        return np.array(Q)

def pR2htm(p, R):
    p_ = np.array(p).ravel()
    if p_.shape[0] == 4:
        p_ = p_[:3]
    R_ = np.array(R)
    H = np.eye(4)
    H[:3, :3] = R_
    H[:3, 3] = p_[:3]
    return H

def progress_bar(i, imax):
    """Prints a progress bar in the terminal.

    Parameters
    ----------
    i : int
        Current iteration.
    imax : int
        Maximum number of iterations.
    """
    sys.stdout.write("\r")
    sys.stdout.write(
        "[%-20s] %d%%" % ("=" * round(20 * i / (imax - 1)), round(100 * i / (imax - 1)))
    )
    sys.stdout.flush()

def D_hat(V, W):
    """The Element-Element (EE-)distance function. This uses the algorithm in
    section 4.1 of the paper, and is an evaluation of ||log(V^-1 W)||_F.

    Parameters
    ----------
    V : np.array
        Homogeneous transformation matrix V (SE(3) element).
    W : np.array
        Homogeneous transformation matrix W (SE(3) element).
    
    Returns
    -------
    distance : float
        The EE-distance between V and W.
    """
    # Compute Z = V^-1 W
    Z = np.linalg.inv(V) @ W
    # Extract the rotation matrix
    Q = Z[:3, :3]
    # Extract the translation vector
    t = Z[:3, 3]
    # Compute the auxiliary variables
    u = 0.5 * (np.trace(Q) - 1) # This is cos(theta)
    v = 1/(2*np.sqrt(2)) * np.linalg.norm(Q - np.linalg.inv(Q), ord='fro') # This is sin(theta)
    # Compute theta with atan2
    theta = np.arctan2(v, u)
    # Recompute cos and theta to ensure correct values
    u = np.cos(theta)
    v = np.sin(theta)
    # Compute alpha
    alpha = (2 - 2*u - theta**2) / (4*(1 - u)**2)
    # Compute matrix M
    M = np.eye(3) * (1 - 2*alpha) + (Q + np.linalg.inv(Q)) * alpha
    # Compute the EE-distance
    distance = np.sqrt(2 * theta**2 + (t.T @ M @ t))
    return distance

def D(V, curve_parametrization):
    """The Element-Curve (EC-)distance function. This uses the brute force
    approach to compute the minimum distance between V and the curve
    parametrization hd(s). (See Definition 3.4 in paper). This function
    also returns the index of the curve that is closest to V for simplicity.

    Parameters
    ----------
    V : np.array
        Homogeneous transformation matrix V (SE(3) element).
    curve_parametrization : np.array
        Array with the precomputed curve. The shape is (n_points, 4, 4).
    
    Returns
    -------
    min_distance : float
        The minimum distance between V and the curve.
    min_index : int
        The index of the curve that is closest to V.
    """
    min_distance = np.inf
    min_index = -1
    for i, Hd_s in enumerate(curve_parametrization):
        distance = D_hat(V, Hd_s)
        if distance < min_distance:
            min_distance = distance
            min_index = i
    return min_distance, min_index

def S(xi):
    """S map in paper (See Definition 2.2). In this case we assume the basis
    as in section 4.3, since it will render the classical twist vector. This
    maps a vector xi into the Lie algebra se(3).

    Parameters
    ----------
    xi : np.array
        Twist vector xi.
    
    Returns
    -------
    lie_algebra : np.array
        The corresponding Lie algebra element.
    """
    xi = np.array(xi).ravel()
    lie_algebra = np.zeros((4, 4)) # Initialize the Lie algebra element
    # The last three elements of xi are assigned to the upper right 3x3 block
    lie_algebra[0, 1] = -xi[5]
    lie_algebra[0, 2] = xi[4]
    lie_algebra[1, 2] = -xi[3]
    # Since the orientation-related portion of the lie algebra is skew-symmetric,
    # we can compute it as (since the position-related portion is still zero):
    lie_algebra = lie_algebra - lie_algebra.T

    # The first three elements of xi are assigned to the last column
    lie_algebra[:3, 3] = xi[:3]
    return lie_algebra

def S_inv(lie_algebra):
    """The inverse of the S map. This maps a Lie algebra element to the twist
    vector. Here we also assume the basis as in section 4.3.

    Parameters
    ----------
    lie_algebra : np.array
        The Lie algebra element.
    
    Returns
    -------
    xi : np.array
        The corresponding twist vector xi.
    """
    xi = np.zeros(6) # Initialize the twist vector
    # The last three elements of xi are assigned from the upper right 3x3 block
    xi[3] = -lie_algebra[1, 2]
    xi[4] = lie_algebra[0, 2]
    xi[5] = -lie_algebra[0, 1]
    # The first three elements of xi are assigned from the last column
    xi[:3] = lie_algebra[:3, 3]
    return (xi).reshape(-1, 1)

def L(f, V, W, arg='V', epsilon=1e-3):
    """L operator (see Definition 2.6). This is a 'derivative' in the Lie group.
    Note that this is a portion of the Lie derivative.

    Parameters
    ----------
    f : function
        Differentiable function that receives two arguments, V and W. This
        function maps SE(3) x SE(3) -> R.
    V : np.array
        Homogeneous transformation matrix V (SE(3) element).
    W : np.array
        Homogeneous transformation matrix W (SE(3) element).
    arg : str or int, optional
        The argument to compute the derivative. The default is 'V'. The options
        are 'V' (0) or 'W' (1). If 'V' is selected, the derivative is computed
        with respect to V, and if 'W' is selected, the derivative is computed
        with respect to W.
    epsilon : float, optional
        Small value to compute the derivative. The default is 1e-3.
    
    Returns
    -------
    L_ : np.array
        The resulting L operator. This is a row vector of shape (1, 6).

    Raises
    ------
    ValueError
        If an invalid argument 'arg' is passed.
    """
    L_ = np.zeros((1, 6)) # Initialize the resulting L
    I = np.eye(6) # Initialize the identity matrix
    if arg.lower() == 'v' or arg == 0:
        curr_f = f(V, W)
        for j in range(6):
            zeta = I[:, j]
            next_f_j = f(expm(S(zeta) * epsilon) @ V, W)
            L_[0, j] = (next_f_j - curr_f) / epsilon
    elif arg.lower() == 'w' or arg == 1:
        curr_f = f(V, W)
        for j in range(6):
            zeta = I[:, j]
            next_f_j = f(V, expm(S(zeta) * epsilon) @ W)
            L_[0, j] = (next_f_j - curr_f) / epsilon
    else:
        raise ValueError("Invalid argument 'arg'. Use 'V' (0) or 'W' (1).")
    return L_

def XI(curve_parametrization, i, ds=0.01):
    """XI operator (See Definition 2.5). This function returns the necessary 
    twist in order to move from the i-th point to the (i+1)-th point in the 
    curve. This is an approximation of the 'derivative' of the curve. Parameter
    'i' will be used as the index of the nearest point in the curve throughout
    the script.

    Parameters
    ----------
    curve_parametrization : np.array
        Array with the precomputed curve. The shape is (n_points, 4, 4).
    i : int
        Index of the point in the curve.
    ds : float, optional
        Small value to compute the variation. The default is 0.01.
    
    Returns
    -------
    XI_ : np.array
        The corresponding twist vector xi.
    """
    # get G
    G = curve_parametrization[i]
    # If the index is the last, then the next point is the first in the curve.
    # Otherwise, the next point is the next in the curve
    if i == len(curve_parametrization) - 1:
        G_next = curve_parametrization[0]
    else:
        G_next = curve_parametrization[i + 1]
    # Compute the 'derivative'
    dGdsigma = (G_next - G) / ds
    XI_ = S_inv(dGdsigma @ np.linalg.inv(G))
    return XI_

def k_N(distance, k1=1.0, k2=20.0):
    """The gain of the normal component (k_N). This is a function of the minimum
    distance between the system and the curve. This is the same function as in
    section 4.4 of the paper.

    Parameters
    ----------
    distance : float
        Minimum distance between the system and the curve.
    k1 : float, optional
        Internal gain of the function. The default is 1.
    k2 : float, optional
        Internal gain of the function. The default is 20.
    
    Returns
    -------
    float
        The gain of the normal component.
    """
    return k1 * np.tanh(k2 * np.sqrt(distance))

def k_T(distance, k1=0.5, k2=1.0, k3=20.0):
    """The gain of the tangent component (k_T). This is a function of the minimum
    distance between the system and the curve. This is the same function as in
    section 4.4 of the paper.

    Parameters
    ----------
    distance : float
        Minimum distance between the system and the curve.
    k1 : float, optional
        Internal gain of the function. The default is 0.5.
    k2 : float, optional
        Internal gain of the function. The default is 1.
    k3 : float, optional
        Internal gain of the function. The default is 20.
    
    Returns
    -------
    float
        The gain of the tangent component.
    """
    return k1 * (1 - k2 * np.tanh(k3 * np.sqrt(distance)))

def gain_factory(gain, *args, **kwargs):
    """Function factory to return the gains with only one argument (distance).
    This is used to simplify the simulation function.

    Parameters
    ----------
    gain : function
        The gain function.
    *args : list
        Arguments of the gain function.
    **kwargs : dict
        Keyword arguments of the gain function.
    
    Returns
    -------
    gain_func : function
        The gain function with only one argument (distance).
    """
    def gain_func(distance):
        return gain(distance, *args, **kwargs)
    return gain_func

R = 0.4
r = 0.1
N_points = 1000
n_frames = 10 # aesthetics only

sphere_center = np.array([-2*R, 0., R])
circle_center = np.array([R, 0, 0.])
htm_sphere = Utils.trn(sphere_center)

# Rotate a reference vector around a given axis, such that the drawn circle
# has radius r
normal = (circle_center - sphere_center) / np.linalg.norm(circle_center - sphere_center)
ref_point = np.array([0, 0, R]).reshape(-1, 1)
ref_point = R * ref_point / np.linalg.norm(ref_point)
phi = np.arccos(r / R)
ref_H = Utils.roty(phi) @ np.array(Utils.trn(ref_point))
theta = np.linspace(0, 2*np.pi, N_points)
rotations = [R_theta(t, normal) for t in theta]
circle_htms = np.array([np.array(Utils.trn(sphere_center) @ Utils.roty(np.deg2rad(-45)) @ Q @ ref_H @ Utils.roty(np.pi)) for Q in rotations])

points = [np.array(H[:3, 3]).ravel() for H in circle_htms]
frames = []
idxs = np.linspace(0, len(circle_htms)-1, 10).astype(int)

for H in np.array(circle_htms)[idxs]:
    frames.append(Frame(htm=H, size=0.1))

circle = PointCloud(points=points, color='cyan', size=0.03)
sphere = Ball(htm=htm_sphere, radius=R, color='magenta', opacity=0.3)

kuka = ub.robot.Robot.create_kuka_kr5()
q_max = kuka.joint_limit[:, 1]
q_min = kuka.joint_limit[:, 0]
sim = ub.Simulation.create_sim_grid([circle, sphere, kuka])
sim.add(frames)

### Control loop
kn2 = 10.0
k_N = gain_factory(k_N, k1=1.0, k2=kn2)
k_T = gain_factory(k_T, k1=0.2, k2=1.0, k3=kn2)
H0 = kuka.fkm().copy()
curve = np.array(circle_htms)
T = 15
dt = 0.01
ds = 1 / len(curve)

imax = int(T/dt)
H_hist = []
distance_hist = []
nearest_point_hist = []
vector_field_hist = []
violations = []
q_hist = []

for i in range(imax):
    progress_bar(i, imax)
    J, H = kuka.jac_geo()
    J = np.array(J).copy()
    H = np.array(H).copy()
    q = np.array(kuka.q.copy())
    violated = (np.nonzero((q > q_max) | (q < q_min))[0]).any()
    violations.append(violated)
    # Compute the minimum distance and the nearest index:
    min_distance, min_index = D(H, curve)
    # Compute the nearest point:
    Hd_star = curve[min_index]
    # Compute the normal component:
    xi_N = -L(D_hat, H, Hd_star, 'V').T
    # Compute the tangent component:
    xi_T = XI(curve, min_index, ds=ds)
    # Compute the vector field:
    psi = k_N(min_distance) * xi_N + k_T(min_distance) * xi_T
    twist_ = psi.copy()
    v = twist_[:3] - np.array(Utils.S(twist_[3:]) @ (H[:3, 3]).reshape(-1, 1)).reshape(-1, 1)
    twist_[:3] = v
    # Compute qdot
    q_dot = np.array(Utils.dp_inv(J) @ twist_)
    kuka.add_ani_frame(time=i * dt, q=q + q_dot * dt)
    H_hist.append(H)
    distance_hist.append(min_distance)
    nearest_point_hist.append(Hd_star)
    vector_field_hist.append(psi)
    q_hist.append(q)

sim.run()

## PLOT CONFIGURATION AND LIMITS

import plotly.graph_objects as go

colorscale = ['#636EFA', '#EF553B', '#00CC96', '#AB63FA', '#FFA15A', '#19D3F3', '#FF6692', '#B6E880', '#FF97FF', '#FECB52']
q_hist_ = np.array(q_hist).reshape(-1, 6)
fig = go.Figure()
time_ = np.arange(len(q_hist_)) * dt

fig.add_trace(go.Scatter(x=time_,
                         y=np.array(violations) * np.max(q_hist_),
                         name='violations', line=dict(width=2, color='red'),
                         fill='tozeroy', fillcolor='rgba(255, 0, 0, 0.1)'),)

for i, qi in enumerate(q_hist_.T):
    fig.add_trace(go.Scatter(x=time_, y=qi, name=f'q{i+1}', mode='lines',
                             line=dict(width=2, color=colorscale[i])),)
    fig.add_trace(go.Scatter(x=time_, y=[q_max[i].item()] * len(time_), name=f'q{i+1}max', mode='lines',
                             line=dict(width=2, color=colorscale[i], dash='dash')),)
    fig.add_trace(go.Scatter(x=time_, y=[q_min[i].item()] * len(time_), name=f'q{i+1}min', mode='lines', 
                             line=dict(width=2, color=colorscale[i], dash='dot'),),)

fig.update_xaxes(title_text="Time (s)", gridcolor='gray', zerolinecolor='gray')
fig.update_yaxes(title_text="Joint position (rad)", gridcolor='gray',
                 zerolinecolor='gray', title_standoff=30)
fig.update_layout(margin=dict(l=20, r=0, t=20, b=0),)


fig.show()

### PLOT TASK FUNCTION

import plotly.graph_objects as go
from plotly.subplots import make_subplots

closest_points = nearest_point_hist
states = H_hist
distances = distance_hist

ori_errs = []
pos_errs = []
# Compute the distance, position error, and orientation error
for closest_point, state in zip(closest_points, states):
    p_near = closest_point[:3, 3]
    ori_near = closest_point[:3, :3]
    p_curr = state[:3, 3]
    ori_curr = state[:3, :3]
    pos_errs.append(np.linalg.norm(p_near - p_curr) * 100)
    trace_ = np.trace(ori_near @ np.linalg.inv(ori_curr))
    acos = np.arccos((trace_ - 1) / 2)
    # checks if acos is nan
    if np.isnan(acos):
        acos = 0
    ori_errs.append(acos * 180 / np.pi)

# Create a figure with three plots, one above another. First the distance, 
# then position error, and the orientation error
time_vec = np.arange(0, len(pos_errs) * dt, dt)
fig = make_subplots(rows=3, cols=1, shared_xaxes=True, vertical_spacing=0.02)
fig.add_trace(go.Scatter(x=time_vec, y=distances, showlegend=False, line=dict(width=3)), row=1, col=1)
fig.add_trace(go.Scatter(x=time_vec, y=pos_errs, showlegend=False, line=dict(width=3)), row=2, col=1)
fig.add_trace(go.Scatter(x=time_vec, y=ori_errs, showlegend=False, line=dict(width=3)), row=3, col=1)
fig.update_xaxes(title_text="Time (s)", gridcolor='gray', zerolinecolor='gray', row=3, col=1)
fig.update_xaxes(title_text="", gridcolor='gray', zerolinecolor='gray', row=1, col=1)
fig.update_xaxes(title_text="", gridcolor='gray', zerolinecolor='gray', row=2, col=1)
fig.update_yaxes(title_text="Distance D", gridcolor='gray', zerolinecolor='gray', 
                 row=1, col=1, title_standoff=30)
fig.update_yaxes(title_text="Pos. error (cm)", gridcolor='gray', zerolinecolor='gray', 
                 row=2, col=1, title_standoff=30)
fig.update_yaxes(title_text="Ori. error (deg)", gridcolor='gray', zerolinecolor='gray', 
                 row=3, col=1, title_standoff=30)

fig.update_layout(width=718.110, height=605.9155)
fig.show()

# Problem 3

In [ ]:
import uaibot as ub
import numpy as np
from uaibot.simobjects import Ball, Frame
from uaibot.utils import Utils


def sphere_collision_models(robot, q):
    """Computes collision models using spheres for Staubli robot at
    configuration 'q'

    Parameters:
    -----------
    robot: uaibot.Robot
        Staubli object
    q: np.npdarray
        The configuration

    Returns:
    --------
    col_models: list
        A list with every collision graph_objects
    """
    q = np.array(q)
    htms = robot.fkm(axis="dh", q=q)
    col_models = []

    # Link 1
    col_model_1 = [
        Ball(htm=htms[0], radius=0.1, opacity=0.5),
        Ball(htm=htms[0] @ Utils.trn([0, -0.07, 0]), radius=0.1, opacity=0.5),
        Ball(htm=htms[0] @ Utils.trn([0, -0.13, 0]), radius=0.1, opacity=0.5),
        Ball(htm=htms[0] @ Utils.trn([0, 0, -0.07]), radius=0.1, opacity=0.5),
        Ball(htm=htms[0] @ Utils.trn([0, 0, -0.13]), radius=0.1, opacity=0.5),
    ]

    # Link 2
    col_model_2 = [
        Ball(
            htm=htms[1] @ Utils.trn([0, 0, -0.07]),
            radius=0.1,
            opacity=0.5,
            color="blue",
        ),
        Ball(
            htm=htms[1] @ Utils.trn([-0.1, 0, -0.12]),
            radius=0.08,
            opacity=0.5,
            color="blue",
        ),
        Ball(
            htm=htms[1] @ Utils.trn([-0.2, 0.04, -0.12]),
            radius=0.08,
            opacity=0.5,
            color="blue",
        ),
        Ball(
            htm=htms[1] @ Utils.trn([-0.2, -0.04, -0.12]),
            radius=0.08,
            opacity=0.5,
            color="blue",
        ),
    ]

    # Link 3
    col_model_3 = [
        Ball(
            htm=htms[2] @ Utils.trn([0, 0, 0.03]),
            radius=0.11,
            opacity=0.5,
            color="green",
        ),
        Ball(
            htm=htms[2] @ Utils.trn([0, -0.03, -0.06]),
            radius=0.11,
            opacity=0.5,
            color="green",
        ),
    ]

    # Link 4
    col_model_4 = [
        Ball(
            htm=htms[3] @ Utils.trn([0, -0.02, 0]),
            radius=0.08,
            opacity=0.5,
            color="magenta",
        ),
        Ball(
            htm=htms[3] @ Utils.trn([0, -0.1, 0]),
            radius=0.08,
            opacity=0.5,
            color="magenta",
        ),
        Ball(
            htm=htms[3] @ Utils.trn([0, -0.15, 0]),
            radius=0.08,
            opacity=0.5,
            color="magenta",
        ),
    ]

    # Link 5
    col_model_5 = [
        Ball(
            htm=htms[4] @ Utils.trn([0, 0.0, 0.04]),
            radius=0.04,
            opacity=0.5,
            color="cyan",
        ),
    ]

    col_models.extend(col_model_1)
    col_models.extend(col_model_2)
    col_models.extend(col_model_3)
    col_models.extend(col_model_4)
    col_models.extend(col_model_5)

    return col_models


rng = np.random.default_rng(42)
q = rng.random(size=(6, 1)) * np.pi
print(q)
staubli = ub.Robot.create_staubli_tx60(opacity=0.7)
# q = staubli.q.copy()
staubli.set_ani_frame(q=q)
staubli.update_col_object(time=0)

sim = ub.simulation.Simulation.create_sim_grid([staubli])
col_models = sphere_collision_models(staubli, q)
# for link in staubli.links:
#     for colobj in link.col_objects:
#         sim.add(colobj[0])
sim.add(col_models)
sim.run()